## Goal:

- Create a map of foreclosures and refinances according to Emma's spreadsheet.

## Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os

## Data read in

In [2]:
refinances = pd.read_clipboard()
refinances.to_csv("refinances.csv")

In [3]:
foreclosures = pd.read_clipboard()
foreclosures.to_csv("foreclosures.csv")

In [4]:
refinances = pd.read_csv("refinances.csv")
refinances = refinances.drop(columns='Unnamed: 0')

foreclosures = pd.read_csv("foreclosures.csv")
foreclosures = foreclosures.drop(columns='Unnamed: 0')

In [5]:
concat_df = pd.concat([refinances,foreclosures])

In [6]:
concat_df['full_address'] = concat_df['address'] + ' IL'

## Geocode

In [7]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [8]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
concat_df['geocoded'] = concat_df['full_address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
concat_df[['lat', 'lon']] = concat_df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
concat_df['lat'] = concat_df['lat'].astype(float)
concat_df['lon'] = concat_df['lon'].astype(float)

# Create the 'geometry' column with Point objects
concat_df['geometry'] = concat_df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
concat_df = concat_df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(concat_df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,building name,borrower,lender,loan amount,address,loan_int,loan_sh,type,full_address,geocoded,lat,lon,geometry
0,The Archer,Strategic Properties of North America,Redwood Trust,$26 million,"1211 North La Salle Street, Chicago",26000000,$26M,Foreclosure,"1211 North La Salle Street, Chicago IL","(41.9042995, -87.6326984)",41.904300,-87.632698,POINT (-87.63270 41.90430)
1,1411 South Michigan,Russland Capital,Ares Management,$80 million,"1411 South Michigan Avenue, Chicago",80000000,$80M,Foreclosure,"1411 South Michigan Avenue, Chicago IL","(41.8637181, -87.623818)",41.863718,-87.623818,POINT (-87.62382 41.86372)
2,21 East Chestnut,**Jos. Cacciatore & Co. Real Estate,HIG,$50 million,"21 East Chestnut Street, Chicago",50000000,$50M,Foreclosure,"21 East Chestnut Street, Chicago IL","(41.898133, -87.62718)",41.898133,-87.627180,POINT (-87.62718 41.89813)
3,The Residences at Payton Place,CA Ventures,TPG Real Estate Finance Trust,$80 million,"3401 West Payton Place, Arlington Heights",80000000,$80M,Foreclosure,"3401 West Payton Place, Arlington Heights IL","(42.0893153, -88.02316909999999)",42.089315,-88.023169,POINT (-88.02317 42.08932)
0,The Archer,Strategic Properties of North America,Redwood Trust,$26 million,"1211 North La Salle Street, Chicago",26000000,$26M,Foreclosure,"1211 North La Salle Street, Chicago IL","(41.9042995, -87.6326984)",41.904300,-87.632698,POINT (-87.63270 41.90430)
1,1411 South Michigan,Russland Capital,Ares Management,$80 million,"1411 South Michigan Avenue, Chicago",80000000,$80M,Foreclosure,"1411 South Michigan Avenue, Chicago IL","(41.8637181, -87.623818)",41.863718,-87.623818,POINT (-87.62382 41.86372)
2,21 East Chestnut,**Jos. Cacciatore & Co. Real Estate,HIG,$50 million,"21 East Chestnut Street, Chicago",50000000,$50M,Foreclosure,"21 East Chestnut Street, Chicago IL","(41.898133, -87.62718)",41.898133,-87.627180,POINT (-87.62718 41.89813)
3,The Residences at Payton Place,CA Ventures,TPG Real Estate Finance Trust,$80 million,"3401 West Payton Place, Arlington Heights",80000000,$80M,Foreclosure,"3401 West Payton Place, Arlington Heights IL","(42.0893153, -88.02316909999999)",42.089315,-88.023169,POINT (-88.02317 42.08932)


In [9]:
print(f'${gdf.loan_int.min():,}')
print('------------------')
print(f'${gdf.loan_int.max():,}')

$26,000,000
------------------
$80,000,000


In [10]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [11]:
gdf.to_file('output.geojson', driver='GeoJSON')

In [12]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/chicago_refinances_foreclosures_07_31_24
